![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# The Role of a Context Engine

## Introduction

A **Context Engine** is the technical infrastructure that powers context engineering. It's the system responsible for storing, retrieving, managing, and serving contextual information to AI agents and applications.

Think of a context engine as the "brain's memory system" - it handles both the storage of information and the intelligent retrieval of relevant context when needed. Just as human memory involves complex processes of encoding, storage, and retrieval, a context engine manages these same processes for AI systems.

## What Makes a Context Engine?

A context engine typically consists of several key components:

### 🗄️ **Storage Layer**
- **Vector databases** for semantic similarity search
- **Traditional databases** for structured data
- **Cache systems** for fast access to frequently used context
- **File systems** for large documents and media

### 🔍 **Retrieval Layer**
- **Semantic search** using embeddings and vector similarity
- **Keyword search** for exact matches and structured queries
- **Hybrid search** combining multiple retrieval methods
- **Ranking algorithms** to prioritize relevant results

### 🧠 **Memory Management**
- **Working memory** for active conversations, sessions, and task-related data (persistent)
- **Long-term memory** for knowledge learned across sessions (user preferences, important facts)
- **Memory consolidation** for moving important information from working to long-term memory

### 🔄 **Integration Layer**
- **APIs** for connecting with AI models and applications
- **Streaming interfaces** for real-time context updates
- **Batch processing** for large-scale context ingestion
- **Event systems** for reactive context management

## Redis as a Context Engine

Redis is uniquely positioned to serve as a context engine because it provides:

- **Vector Search**: Native support for semantic similarity search
- **Multiple Data Types**: JSON documents, strings, hashes, lists, sets, streams, and more
- **High Performance**: In-memory processing with sub-millisecond latency
- **Persistence**: Durable storage with various persistence options
- **Scalability**: Horizontal scaling with Redis Cluster
- **Rich Ecosystem**: Integrations with AI frameworks and tools

Let's explore how Redis functions as a context engine in our university class agent.

In [ ]:
# Install the Redis Context Course package
%pip install -q -e ../../reference-agent

In [ ]:
import os
import json
import numpy as np
import sys
from typing import List, Dict, Any

# Set up environment - handle both interactive and CI environments
def _set_env(key: str):
    if key not in os.environ:
        # Check if we're in an interactive environment
        if hasattr(sys.stdin, 'isatty') and sys.stdin.isatty():
            import getpass
            os.environ[key] = getpass.getpass(f"{key}: ")
        else:
            # Non-interactive environment (like CI) - use a dummy key
            print(f"⚠️  Non-interactive environment detected. Using dummy {key} for demonstration.")
            os.environ[key] = "sk-dummy-key-for-testing-purposes-only"

_set_env("OPENAI_API_KEY")
os.environ["REDIS_URL"] = "redis://localhost:6379"

## Context Engine Architecture

Let's examine the architecture of our Redis-based context engine:

In [ ]:
# Import Redis Context Course components with error handling
try:
    from redis_context_course.redis_config import redis_config
    from redis_context_course import MemoryClient
    from redis_context_course.course_manager import CourseManager
    import redis
    
    PACKAGE_AVAILABLE = True
    print("✅ Redis Context Course package imported successfully")
    
    # Check Redis connection
    redis_healthy = redis_config.health_check()
    print(f"📡 Redis Connection: {'✅ Healthy' if redis_healthy else '❌ Failed'}")
    
    if redis_healthy:
        # Show Redis info
        redis_info = redis_config.redis_client.info()
        print(f"📊 Redis Version: {redis_info.get('redis_version', 'Unknown')}")
        print(f"💾 Memory Usage: {redis_info.get('used_memory_human', 'Unknown')}")
        print(f"🔗 Connected Clients: {redis_info.get('connected_clients', 'Unknown')}")
        
        # Show configured indexes
        print(f"\n🗂️ Vector Indexes:")
        print(f"  • Course Catalog: {redis_config.vector_index_name}")
        print(f"  • Agent Memory: Managed by Agent Memory Server")
        
        # Show data types in use
        print(f"\n📋 Data Types in Use:")
        print(f"  • Hashes: Course storage")
        print(f"  • Vectors: Semantic embeddings (1536 dimensions)")
        print(f"  • Strings: Simple key-value pairs")
        print(f"  • Sets: Tags and categories")
    
except ImportError as e:
    print(f"⚠️  Package not available: {e}")
    print("📝 This is expected in CI environments. Creating mock objects for demonstration...")
    
    # Create mock classes
    class MockRedisConfig:
        def __init__(self):
            self.vector_index_name = "course_catalog_index"
        
        def health_check(self):
            return False  # Simulate Redis not available in CI
    
    class MemoryClient:
        def __init__(self, student_id: str):
            self.student_id = student_id
            print(f"📝 Mock MemoryClient created for {student_id}")
        
        async def store_memory(self, content: str, memory_type: str, importance: float = 0.5, metadata: dict = None):
            return "mock-memory-id-12345"
        
        async def retrieve_memories(self, query: str, limit: int = 5):
            class MockMemory:
                def __init__(self, content: str, memory_type: str):
                    self.content = content
                    self.memory_type = memory_type
            
            return [
                MockMemory("Student prefers online courses", "preference"),
                MockMemory("Goal: AI specialization", "goal"),
                MockMemory("Strong programming background", "academic_performance")
            ]
        
        async def get_student_context(self, query: str):
            return {
                "preferences": ["online courses", "flexible schedule"],
                "goals": ["machine learning specialization"],
                "general_memories": ["programming experience"],
                "recent_conversations": ["course planning session"]
            }
    
    class CourseManager:
        def __init__(self):
            print("📝 Mock CourseManager created")
    
    redis_config = MockRedisConfig()
    redis_healthy = False
    PACKAGE_AVAILABLE = False
    print("✅ Mock objects created for demonstration")

# Initialize our context engine components
print("\n🏗️ Context Engine Architecture")
print("=" * 50)
print(f"📡 Redis Connection: {'✅ Healthy' if redis_healthy else '❌ Failed (using mock data)'}")

## Storage Layer Deep Dive

Let's explore how different types of context are stored in Redis:

In [ ]:
# Demonstrate different storage patterns
print("💾 Storage Layer Patterns")
print("=" * 40)

# 1. Structured Data Storage (Hashes)
print("\n1️⃣ Structured Data (Redis Hashes)")
sample_course_data = {
    "course_code": "CS101",
    "title": "Introduction to Programming",
    "credits": "3",
    "department": "Computer Science",
    "difficulty_level": "beginner",
    "format": "online"
}

print("Course data stored as hash:")
for key, value in sample_course_data.items():
    print(f"  {key}: {value}")

# 2. Vector Storage for Semantic Search
print("\n2️⃣ Vector Embeddings (1536-dimensional)")
print("Sample embedding vector (first 10 dimensions):")
sample_embedding = np.random.rand(10)  # Simulated embedding
print(f"  [{', '.join([f'{x:.4f}' for x in sample_embedding])}...]")
print(f"  Full vector: 1536 dimensions, stored as binary data")

# 3. Memory Storage Patterns
print("\n3️⃣ Memory Storage (Timestamped Records)")
sample_memory = {
    "id": "mem_12345",
    "student_id": "student_alex",
    "content": "Student prefers online courses due to work schedule",
    "memory_type": "preference",
    "importance": "0.9",
    "created_at": "1703123456.789",
    "metadata": '{"context": "course_planning"}'
}

print("Memory record structure:")
for key, value in sample_memory.items():
    print(f"  {key}: {value}")

## Retrieval Layer in Action

The retrieval layer is where the magic happens - turning queries into relevant context:

In [ ]:
# Demonstrate different retrieval methods
print("🔍 Retrieval Layer Methods")
print("=" * 40)

# Initialize managers
import os
from agent_memory_client import MemoryClientConfig

config = MemoryClientConfig(
    base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8000"),
    default_namespace="redis_university"
)
memory_client = MemoryClient(config=config)
course_manager = CourseManager()

async def demonstrate_retrieval_methods():
    # 1. Exact Match Retrieval
    print("\n1️⃣ Exact Match Retrieval")
    print("Query: Find course with code 'CS101'")
    print("Method: Direct key lookup or tag filter")
    print("Use case: Looking up specific courses, IDs, or codes")
    
    # 2. Semantic Similarity Search
    print("\n2️⃣ Semantic Similarity Search")
    print("Query: 'I want to learn machine learning'")
    print("Process:")
    print("  1. Convert query to embedding vector")
    print("  2. Calculate cosine similarity with stored vectors")
    print("  3. Return top-k most similar results")
    print("  4. Apply similarity threshold filtering")
    
    # Simulate semantic search process
    query = "machine learning courses"
    print(f"\n🔍 Simulating semantic search for: '{query}'")
    
    # This would normally generate an actual embedding
    print("  Step 1: Generate query embedding... ✅")
    print("  Step 2: Search vector index... ✅")
    print("  Step 3: Calculate similarities... ✅")
    print("  Step 4: Rank and filter results... ✅")
    
    # 3. Hybrid Search
    print("\n3️⃣ Hybrid Search (Semantic + Filters)")
    print("Query: 'online programming courses for beginners'")
    print("Process:")
    print("  1. Semantic search: 'programming courses'")
    print("  2. Apply filters: format='online', difficulty='beginner'")
    print("  3. Combine and rank results")
    
    # 4. Memory Retrieval
    print("\n4️⃣ Memory Retrieval")
    print("Query: 'What are my course preferences?'")
    print("Process:")
    print("  1. Semantic search in memory index")
    print("  2. Filter by memory_type='preference'")
    print("  3. Sort by importance and recency")
    print("  4. Return relevant memories")

await demonstrate_retrieval_methods()

## Memory Management System

Let's explore how the context engine manages different types of memory:

In [ ]:
# Demonstrate memory management
print("🧠 Memory Management System")
print("=" * 40)

async def demonstrate_memory_management():
    # Working Memory (Task-Focused Context)
    print("\n📝 Working Memory (Persistent Task Context)")
    print("Purpose: Maintain conversation flow and task-related data")
    print("Storage: Redis Streams and Hashes (LangGraph Checkpointer)")
    print("Lifecycle: Persistent during task, can span multiple sessions")
    print("Example data:")
    print("  • Current conversation messages")
    print("  • Agent state and workflow position")
    print("  • Task-related variables and computations")
    print("  • Tool call results and intermediate steps")
    print("  • Search results being processed")
    print("  • Cached embeddings for current task")
    
    # Long-term Memory (Cross-Session Knowledge)
    print("\n🗄️ Long-term Memory (Cross-Session Knowledge)")
    print("Purpose: Store knowledge learned across sessions")
    print("Storage: Redis Vector Index with embeddings")
    print("Lifecycle: Persistent across all sessions")
    print("Example data:")
    
    # Store some example memories
    memory_examples = [
        ("preference", "Student prefers online courses", 0.9),
        ("goal", "Wants to specialize in AI and machine learning", 1.0),
        ("experience", "Struggled with calculus but excelled in programming", 0.8),
        ("context", "Works part-time, needs flexible schedule", 0.7)
    ]
    
    for memory_type, content, importance in memory_examples:
        print(f"  • [{memory_type.upper()}] {content} (importance: {importance})")
    
    # Memory Consolidation
    print("\n🔄 Memory Consolidation Process")
    print("Purpose: Move important information from working to long-term memory")
    print("Triggers:")
    print("  • Conversation length exceeds threshold (20+ messages)")
    print("  • Important preferences or goals mentioned")
    print("  • Significant events or decisions made")
    print("  • End of session or explicit save commands")
    
    print("\n📊 Memory Status (Conceptual):")
    print(f"  • Preferences stored: 1 (online courses)")
    print(f"  • Goals stored: 1 (AI/ML specialization)")
    print(f"  • General memories: 2 (calculus struggle, part-time work)")
    print(f"  • Conversation summaries: 0 (new session)")
    print("\nNote: See Section 3 notebooks for actual memory implementation.")

await demonstrate_memory_management()

## Integration Layer: Connecting Everything

The integration layer is how the context engine connects with AI models and applications:

In [ ]:
# Demonstrate integration patterns
print("🔄 Integration Layer Patterns")
print("=" * 40)

# 1. LangGraph Integration
print("\n1️⃣ LangGraph Integration (Checkpointer)")
print("Purpose: Persistent agent state and conversation history")
print("Pattern: Redis as state store for workflow nodes")
print("Benefits:")
print("  • Automatic state persistence")
print("  • Resume conversations across sessions")
print("  • Parallel execution support")
print("  • Built-in error recovery")

# Show checkpointer configuration
checkpointer_config = {
    "redis_client": "Connected Redis instance",
    "namespace": "class_agent",
    "serialization": "JSON with binary support",
    "key_pattern": "namespace:thread_id:checkpoint_id"
}

print("\nCheckpointer Configuration:")
for key, value in checkpointer_config.items():
    print(f"  {key}: {value}")

# 2. OpenAI Integration
print("\n2️⃣ OpenAI Integration (Embeddings & Chat)")
print("Purpose: Generate embeddings and chat completions")
print("Pattern: Context engine provides relevant information to LLM")
print("Flow:")
print("  1. User query → Context engine retrieval")
print("  2. Retrieved context → System prompt construction")
print("  3. Enhanced prompt → OpenAI API")
print("  4. LLM response → Context engine storage")

# 3. Tool Integration
print("\n3️⃣ Tool Integration (LangChain Tools)")
print("Purpose: Expose context engine capabilities as agent tools")
print("Available tools:")
tools_info = [
    ("search_courses_tool", "Semantic search in course catalog"),
    ("get_recommendations_tool", "Personalized course recommendations"),
    ("store_preference_tool", "Save user preferences to memory"),
    ("store_goal_tool", "Save user goals to memory"),
    ("get_student_context_tool", "Retrieve relevant user context")
]

for tool_name, description in tools_info:
    print(f"  • {tool_name}: {description}")

## Performance Characteristics

Let's examine the performance characteristics of our Redis-based context engine:

**Conceptual Example (not executable in this notebook)**

```python
import time
import asyncio

# Performance benchmarking
print("⚡ Performance Characteristics")
print("=" * 40)

async def benchmark_context_engine():
    # 1. Memory Storage Performance
    print("\n📝 Memory Storage Performance")
    start_time = time.time()
    
    # Store multiple memories
    memory_tasks = []
    for i in range(10):
#         task = memory_manager.store_memory(
            f"Test memory {i} for performance benchmarking",
            "benchmark",
            importance=0.5
        )
        memory_tasks.append(task)
    
    await asyncio.gather(*memory_tasks)
    storage_time = time.time() - start_time
    
    print(f"  Stored 10 memories in {storage_time:.3f} seconds")
    print(f"  Average: {(storage_time/10)*1000:.1f} ms per memory")
    
    # 2. Memory Retrieval Performance
    print("\n🔍 Memory Retrieval Performance")
    start_time = time.time()
    
    # Perform multiple retrievals
    retrieval_tasks = []
    for i in range(5):
#         task = memory_manager.retrieve_memories(
            f"performance test query {i}",
            limit=5
        )
        retrieval_tasks.append(task)
    
    results = await asyncio.gather(*retrieval_tasks)
    retrieval_time = time.time() - start_time
    
    total_results = sum(len(result) for result in results)
    print(f"  Retrieved {total_results} memories in {retrieval_time:.3f} seconds")
    print(f"  Average: {(retrieval_time/5)*1000:.1f} ms per query")
    
    # 3. Context Integration Performance
    print("\n🧠 Context Integration Performance")
    start_time = time.time()
    
    # Get comprehensive student context
#     context = await memory_manager.get_student_context(
        "comprehensive context for performance testing"
    )
    
    integration_time = time.time() - start_time
    context_size = len(str(context))
    
    print(f"  Integrated context in {integration_time:.3f} seconds")
    print(f"  Context size: {context_size} characters")
    print(f"  Throughput: {context_size/integration_time:.0f} chars/second")

# Run performance benchmark
if redis_config.health_check():
    await benchmark_context_engine()
else:
    print("❌ Redis not available for performance testing")```

*Note: This demonstrates the concept. See Section 3 notebooks for actual memory implementation using MemoryClient.*


## Context Engine Best Practices

Based on our implementation, here are key best practices for building context engines:

In [ ]:
# Best practices demonstration
print("💡 Context Engine Best Practices")
print("=" * 50)

print("\n1️⃣ **Data Organization**")
print("✅ Use consistent naming conventions for keys")
print("✅ Separate different data types into different indexes")
print("✅ Include metadata for filtering and sorting")
print("✅ Use appropriate data structures for each use case")

print("\n2️⃣ **Memory Management**")
print("✅ Implement memory consolidation strategies")
print("✅ Use importance scoring for memory prioritization")
print("✅ Distinguish between working memory (task-focused) and long-term memory (cross-session)")
print("✅ Monitor memory usage and implement cleanup")

print("\n3️⃣ **Search Optimization**")
print("✅ Use appropriate similarity thresholds")
print("✅ Combine semantic and keyword search when needed")
print("✅ Implement result ranking and filtering")
print("✅ Cache frequently accessed embeddings")

print("\n4️⃣ **Performance Optimization**")
print("✅ Use connection pooling for Redis clients")
print("✅ Batch operations when possible")
print("✅ Implement async operations for I/O")
print("✅ Monitor and optimize query performance")

print("\n5️⃣ **Error Handling**")
print("✅ Implement graceful degradation")
print("✅ Use circuit breakers for external services")
print("✅ Log errors with sufficient context")
print("✅ Provide fallback mechanisms")

print("\n6️⃣ **Security & Privacy**")
print("✅ Encrypt sensitive data at rest")
print("✅ Use secure connections (TLS)")
print("✅ Implement proper access controls")
print("✅ Anonymize or pseudonymize personal data")

# Show example of good key naming
print("\n📝 Example: Good Key Naming Convention")
key_examples = [
    "course_catalog:CS101",
    "agent_memory:student_alex:preference:mem_12345",
    "session:thread_abc123:checkpoint:step_5",
    "cache:embedding:query_hash_xyz789"
]

for key in key_examples:
    print(f"  {key}")
    
print("\nPattern: namespace:entity:type:identifier")

## Real-World Context Engine Example

Let's see our context engine in action with a realistic scenario:

**Conceptual Example (not executable in this notebook)**

```python
# Real-world scenario demonstration
print("🌍 Real-World Context Engine Scenario")
print("=" * 50)

async def realistic_scenario():
    print("\n📚 Scenario: Student Planning Next Semester")
    print("-" * 40)
    
    # Step 1: Student context retrieval
    print("\n1️⃣ Context Retrieval Phase")
    query = "I need help planning my courses for next semester"
    print(f"Student Query: '{query}'")
    
    # Simulate context retrieval
    print("\n🔍 Context Engine Processing:")
    print("  • Retrieving student profile...")
    print("  • Searching relevant memories...")
    print("  • Loading academic history...")
    print("  • Checking preferences and goals...")
    
    # Get actual context
#     context = await memory_manager.get_student_context(query)
    
    print("\n📋 Retrieved Context:")
    print(f"  • Preferences: {len(context.get('preferences', []))} stored")
    print(f"  • Goals: {len(context.get('goals', []))} stored")
    print(f"  • Conversation history: {len(context.get('recent_conversations', []))} summaries")
    
    # Step 2: Context integration
    print("\n2️⃣ Context Integration Phase")
    print("🧠 Integrating multiple context sources:")
    
    integrated_context = {
        "student_profile": {
            "major": "Computer Science",
            "year": 2,
            "completed_credits": 45,
            "gpa": 3.7
        },
        "preferences": [
            "Prefers online courses due to work schedule",
            "Interested in machine learning and AI",
            "Wants hands-on programming experience"
        ],
        "constraints": [
            "Maximum 15 credits per semester",
            "Must complete CS201 prerequisite",
            "Available Tuesday/Thursday evenings"
        ],
        "goals": [
            "Graduate in 4 years",
            "Specialize in AI/ML",
            "Maintain 3.5+ GPA"
        ]
    }
    
    for category, items in integrated_context.items():
        print(f"  • {category.title()}: {len(items) if isinstance(items, list) else 'Profile loaded'}")
    
    # Step 3: Intelligent response generation
    print("\n3️⃣ Response Generation Phase")
    print("🤖 Context-aware response:")
    print("-" * 30)
    
    response = f"""
Based on your profile and our previous conversations, here's my recommendation for next semester:

🎯 **Personalized Plan for CS Year 2 Student:**

**Recommended Courses (12 credits):**
1. **CS301: Machine Learning Fundamentals** (4 credits, Online)
   → Aligns with your AI specialization goal
   → Available Tuesday evenings (fits your schedule)
   → Prerequisite CS201 will be completed this semester

2. **CS250: Database Systems** (4 credits, Hybrid)
   → Essential for CS major requirements
   → Practical skills valuable for internships
   → Thursday evening lab sessions

3. **MATH301: Statistics** (4 credits, Online)
   → Required for ML specialization
   → Fully online (matches your preference)
   → Self-paced with flexible deadlines

**Why this plan works:**
✅ Stays within your 15-credit limit
✅ All courses available in preferred formats
✅ Fits your Tuesday/Thursday availability
✅ Advances your AI/ML specialization goal
✅ Maintains manageable workload for 3.5+ GPA

**Next steps:**
1. Verify CS201 completion this semester
2. Check for any schedule conflicts
3. Register early - these courses fill up quickly!

Would you like me to help you explore any of these courses in more detail?
"""
    
    print(response)
    
    # Step 4: Memory consolidation
    print("\n4️⃣ Memory Consolidation Phase")
    print("💾 Storing interaction for future reference:")
    
    # Store the planning session as a memory
#     planning_memory = await memory_manager.store_memory(
        "Student requested semester planning help. Recommended CS301, CS250, MATH301 based on AI/ML goals and schedule constraints.",
        "planning_session",
        importance=0.9,
        metadata={"semester": "Spring 2024", "credits_planned": 12}
    )
    
    print(f"  ✅ Planning session stored (ID: {planning_memory[:8]}...)")
    print("  ✅ Course preferences updated")
    print("  ✅ Academic goals reinforced")
    print("  ✅ Context ready for future interactions")

# Run the realistic scenario
if redis_config.health_check():
    await realistic_scenario()
else:
    print("❌ Redis not available for scenario demonstration")```

*Note: This demonstrates the concept. See Section 3 notebooks for actual memory implementation using MemoryClient.*


## Key Takeaways

From our exploration of context engines, several important principles emerge:

### 1. **Multi-Layer Architecture**
- **Storage Layer**: Handles different data types and access patterns
- **Retrieval Layer**: Provides intelligent search and ranking
- **Memory Management**: Orchestrates working memory (task-focused) and long-term memory (cross-session)
- **Integration Layer**: Connects with AI models and applications

### 2. **Performance is Critical**
- Context retrieval must be fast (< 100ms for good UX)
- Memory storage should be efficient and scalable
- Caching strategies are essential for frequently accessed data
- Async operations prevent blocking in AI workflows

### 3. **Context Quality Matters**
- Relevant context improves AI responses dramatically
- Irrelevant context can confuse or mislead AI models
- Context ranking and filtering are as important as retrieval
- Memory consolidation helps maintain context quality by moving important information to long-term storage

### 4. **Integration is Key**
- Context engines must integrate seamlessly with AI frameworks
- Tool-based integration provides flexibility and modularity
- State management integration enables persistent conversations
- API design affects ease of use and adoption

## Next Steps

In the next section, we'll dive into **Setting up System Context** - how to define what your AI agent should know about itself, its capabilities, and its operating environment. We'll cover:

- System prompt engineering
- Tool definition and management
- Capability boundaries and constraints
- Domain knowledge integration

## Try It Yourself

Experiment with the context engine concepts:

1. **Modify retrieval parameters** - Change similarity thresholds and see how it affects results
2. **Add new memory types** - Create custom memory categories for your use case
3. **Experiment with context integration** - Try different ways of combining context sources
4. **Measure performance** - Benchmark different operations and optimize bottlenecks

The context engine is the foundation that makes sophisticated AI agents possible. Understanding its architecture and capabilities is essential for building effective context engineering solutions.